# Вебинар 4. Домашнее задание

# Домашнее задание

1. (+)Изучите структуру модуля src
2. (+)Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. (+)Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. (+)Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их в качетсве методов класса.  
Посчитайте с помощью них рекоменадции и найдите precision@5
5. (+)Проверьте, что все модули корректно импортируются
6. (+)Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

__________________________________

### Выводы и трудности по домашке

1. Захотелось сделать "более универсальный код". Т.е. учитывать тот факт, чтобы пользователю не выдавались в рекомендациях 2 одинаковых товара, если он попал в топ у двух похожих пользователей, например. В итоге накрутил дополнительных циклов в процедурах с проверками, искать следующий, если такой уже есть. Но по итогу пришлось подпереть несколькими "костылями". Оказалось, что для некоторых пользователей может вылетать 3 рекомендации или 0. А мы хотим предсказывать 5 рекомендаций. Наверное, это связано с тем, что было мало покупок. Или с одним параметром,  о котором будет ниже. В общем пришлось учесть и эти варианты. Код получился так себе, для прода его однозначно надо писать иначе

2. Различные варианты префильтра наших данных я реализовал, они работают. Но в конечной версии здесь в домашке выключил все при вызове этой процедуры, кроме нашего привычного Топ5000. Связано это было с поиском причины, описанной в следующем пункте. А времени, на новые опыты больше не осталось ((

3. Параметр filter_already_liked_items у ItemRecomender оказался прям откровением! Мы как-то "молча" включили его в этом вебинаре и я долго не мог понять, почему так отличается скорр на прошлых уроках и здесь. Оказалось он влияет очень сильно. Писал об этом в чате. Если включить этот параметр, метрика падает до уровня рандомных рекомендаций. Если выключить, получается очень неплохой. В конце методички представлен именно вариант с выключенным параметром filter_already_liked_items и результаты

4. Странное поведение наблюдал при взвшенивании матрицы алгоритмом bm25 и без взвешивания. Если матрицу не взвешивать скорость предсказания падает почему-то в разы! Не метрика, а именно скорость работы алгоритма предсказания. Обучение скорость не меняется особо. Если взвешивать, то скорость хорошая. Возможно это тоже связано с тем как "я готовлю данные" в методе "_init_".

5. В целом домашка отняла больше времени, но связано это было с недостаточной практикой в работе с кодом. Почему не запускаетя? Почему тут ошибка? И с подобными проблемами. Но думаю, это важно, проходить через эти трудности самому

____________________

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../Lesson2/data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../Lesson2/data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

filtered_items = item_features.loc[item_features['department'] == 'MISC. TRANS.', 'item_id'].tolist()

data_train = prefilter_items(data_train, filter_top_popular=5000,
                                exclude_top_popular=None, 
                                exclude_top_unpopular=None,
                                exclude_not_sold_weeks=None,
                                exclude_items=None,
                                exclude_cheaper_than = None,
                                exclude_more_expensive_than=None
                               )

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


-----

In [7]:
model = MainRecommender(user_item_matrix, weighting=True, show_progress=True) 

In [8]:
result['own_recommends'] = result.apply(lambda row: model.get_own_recommends(user_id=row['user_id']), axis=1)

In [9]:
%time
result['similar_items_recommends'] = result.apply(lambda row: model.get_similar_items_recommendation(user_id=row['user_id']), axis=1)

Wall time: 0 ns
Error: Рекоммендаций меньше, чем ожидается. user_id: 75
Error: Рекоммендаций меньше, чем ожидается. user_id: 650
Error: Рекоммендаций меньше, чем ожидается. user_id: 729
Error: Рекоммендаций меньше, чем ожидается. user_id: 954
Error: Рекоммендаций меньше, чем ожидается. user_id: 1105
Error: Рекоммендаций меньше, чем ожидается. user_id: 1539
Error: Рекоммендаций меньше, чем ожидается. user_id: 1565
Error: Рекоммендаций меньше, чем ожидается. user_id: 1636
Error: Рекоммендаций меньше, чем ожидается. user_id: 1893
Error: Рекоммендаций меньше, чем ожидается. user_id: 1984
Error: Рекоммендаций меньше, чем ожидается. user_id: 1987
Error: Рекоммендаций меньше, чем ожидается. user_id: 2120
Error: Рекоммендаций меньше, чем ожидается. user_id: 2259
Error: Рекоммендаций меньше, чем ожидается. user_id: 2355
Error: Рекоммендаций меньше, чем ожидается. user_id: 2364
Error: Рекоммендаций меньше, чем ожидается. user_id: 2373
Error: Рекоммендаций меньше, чем ожидается. user_id: 2404


In [10]:
%time
result['similar_users_recommends'] = result.apply(lambda row: model.get_similar_users_recommendation(user_id=row['user_id']), axis=1)


Wall time: 0 ns
Error: Рекоммендаций меньше, чем ожидается. user_id: 13
Error: Рекоммендаций меньше, чем ожидается. user_id: 33
Error: Рекоммендаций меньше, чем ожидается. user_id: 58
Error: Рекоммендаций меньше, чем ожидается. user_id: 63
Error: Рекоммендаций меньше, чем ожидается. user_id: 68
Error: Рекоммендаций меньше, чем ожидается. user_id: 75
Error: Рекоммендаций меньше, чем ожидается. user_id: 125
Error: Рекоммендаций меньше, чем ожидается. user_id: 155
Error: Рекоммендаций меньше, чем ожидается. user_id: 183
Error: Рекоммендаций меньше, чем ожидается. user_id: 194
Error: Рекоммендаций меньше, чем ожидается. user_id: 205
Error: Рекоммендаций меньше, чем ожидается. user_id: 226
Error: Рекоммендаций меньше, чем ожидается. user_id: 230
Error: Рекоммендаций меньше, чем ожидается. user_id: 240
Error: Рекоммендаций меньше, чем ожидается. user_id: 245
Error: Рекоммендаций меньше, чем ожидается. user_id: 249
Error: Рекоммендаций меньше, чем ожидается. user_id: 251
Error: Рекоммендаций 

Error: Рекоммендаций меньше, чем ожидается. user_id: 1475
Error: Рекоммендаций меньше, чем ожидается. user_id: 1487
Error: Рекоммендаций меньше, чем ожидается. user_id: 1489
Error: Рекоммендаций меньше, чем ожидается. user_id: 1491
Error: Рекоммендаций меньше, чем ожидается. user_id: 1496
Error: Рекоммендаций меньше, чем ожидается. user_id: 1505
Error: Рекоммендаций меньше, чем ожидается. user_id: 1508
Error: Рекоммендаций меньше, чем ожидается. user_id: 1519
Error: Рекоммендаций меньше, чем ожидается. user_id: 1525
Error: Рекоммендаций меньше, чем ожидается. user_id: 1534
Error: Рекоммендаций меньше, чем ожидается. user_id: 1535
Error: Рекоммендаций меньше, чем ожидается. user_id: 1536
Error: Рекоммендаций меньше, чем ожидается. user_id: 1556
Error: Рекоммендаций меньше, чем ожидается. user_id: 1557
Error: Рекоммендаций меньше, чем ожидается. user_id: 1565
Error: Рекоммендаций меньше, чем ожидается. user_id: 1572
Error: Рекоммендаций меньше, чем ожидается. user_id: 1575
Error: Рекомме

In [11]:
result.head(2)

,user_id,actual,own_recommends,similar_items_recommends,similar_users_recommends
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527290, 1043064, 1082185, 1075074, 10149640]","[15926844, 845262, 866211, 993441, 13877192]","[9527290, 1056509, 1098066, 1023958, 1070820]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092026, 1053690, 9527494, 1092937, 9337581]","[1076875, 1092026, 9527160, 832442, 1008892]","[1092026, 1053690, 1110764, 1113588, 995785]"


In [12]:
result.apply(lambda row: precision_at_k(row['own_recommends'], row['actual']), axis=1).mean()

C:\Users\avale\jupiter_notebooks\gb3sm-recomended-systems\Lesson4\src\metrics.py:24: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.28815142576204394

In [13]:
result.apply(lambda row: precision_at_k(row['similar_items_recommends'], row['actual']), axis=1).mean()

0.07079646017699134

In [14]:
result.head(20).apply(lambda row: precision_at_k(row['similar_users_recommends'], row['actual']), axis=1).mean()

0.12250000000000003